In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("online_retail.csv", encoding="ISO-8859-1")

In [3]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [5]:
df.shape

(541909, 8)

In [8]:
df.columns = df.columns.str.strip().str.replace(" ","_").str.lower()

In [7]:
df.columns

Index(['invoiceno', 'stockcode', 'description', 'quantity', 'invoicedate',
       'unitprice', 'customerid', 'country'],
      dtype='object')

In [9]:
df['invoicedate'] = pd.to_datetime(df['invoicedate'], errors="coerce")
df['quantity'] = pd.to_numeric(df['quantity'], errors="coerce")
df['unitprice'] = pd.to_numeric(df['unitprice'], errors="coerce")
df['customerid'] = df['customerid'].astype("string")

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   invoiceno    541909 non-null  object        
 1   stockcode    541909 non-null  object        
 2   description  540455 non-null  object        
 3   quantity     541909 non-null  int64         
 4   invoicedate  541909 non-null  datetime64[ns]
 5   unitprice    541909 non-null  float64       
 6   customerid   406829 non-null  string        
 7   country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(4), string(1)
memory usage: 33.1+ MB


In [11]:
df.shape

(541909, 8)

In [13]:
df = df[df['unitprice']>0]
df = df[df['quantity'] != 0]

In [14]:
df.shape

(539392, 8)

In [15]:
df = df.dropna(subset=["customerid"])

In [16]:
df.shape

(406789, 8)

In [18]:
df["is_return"] = (
    (df['quantity']<0) |
    (df['invoiceno'].astype(str).str.startswith("C"))
)
df['is_return'].value_counts()

,count
is_return,
False,397884
True,8905


In [19]:
df["revenue"] = df["quantity"] * df['unitprice']
df["revenue_abs"] = df['revenue'].abs()

In [20]:
df.shape

(406789, 11)

In [21]:
orders = df.groupby('invoiceno').agg(
    invoice_date = ("invoicedate", "min"),
    customerid = ("customerid", "first"),
    country = ("country", "first"),
    total_revenue = ("revenue", "sum"),
    is_return = ("is_return", "max")
).reset_index()

In [22]:
orders.head()

,invoiceno,invoice_date,customerid,country,total_revenue,is_return
0,536365,2010-12-01 08:26:00,17850.0,United Kingdom,139.12,False
1,536366,2010-12-01 08:28:00,17850.0,United Kingdom,22.20,False
2,536367,2010-12-01 08:34:00,13047.0,United Kingdom,278.73,False
3,536368,2010-12-01 08:34:00,13047.0,United Kingdom,70.05,False
4,536369,2010-12-01 08:35:00,13047.0,United Kingdom,17.85,False


In [23]:
orders.shape

(22186, 6)

In [24]:
orders["year_month"] = orders['invoice_date'].dt.to_period("M").astype(str)

In [25]:
orders["is_return"].value_counts()

,count
is_return,
False,18532
True,3654


In [26]:
sales_df =df[df['is_return']==False].copy()

In [27]:
sales_df.shape

(397884, 11)

In [28]:
reference_date = sales_df['invoicedate'].max() + pd.Timedelta(days=1)
reference_date

Timestamp('2011-12-10 12:50:00')

In [29]:
rfm = sales_df.groupby("customerid").agg(
    recency = ("invoicedate", lambda x: (reference_date-x.max()).days),
    frequency = ("invoiceno", "nunique"),
    monetary = ("revenue", "sum")
).reset_index()

In [30]:
rfm.head()

,customerid,recency,frequency,monetary
0,12346.0,326,1,77183.60
1,12347.0,2,7,4310.00
2,12348.0,75,4,1797.24
3,12349.0,19,1,1757.55
4,12350.0,310,1,334.40


In [31]:
rfm.shape

(4338, 4)

In [32]:
rfm.describe()

,recency,frequency,monetary
count,4338.000000,4338.000000,4338.000000
mean,92.536422,4.272015,2054.266460
std,100.014169,7.697998,8989.230441
min,1.000000,1.000000,3.750000
25%,18.000000,1.000000,307.415000
50%,51.000000,2.000000,674.485000
75%,142.000000,5.000000,1661.740000
max,374.000000,209.000000,280206.020000


In [33]:
rfm["r_score"] = pd.qcut(rfm["recency"], 5, labels=[5,4,3,2,1])
rfm["f_score"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, labels=[1,2,3,4,5])
rfm["m_score"] = pd.qcut(rfm["monetary"], 5, labels=[1,2,3,4,5])

In [34]:
rfm["rfm_score"] = (
    rfm['r_score'].astype(str) +
    rfm['f_score'].astype(str) +
    rfm['m_score'].astype(str)
)

In [35]:
rfm.head()

,customerid,recency,frequency,monetary,r_score,f_score,m_score,rfm_score
0,12346.0,326,1,77183.60,1,1,5,115
1,12347.0,2,7,4310.00,5,5,5,555
2,12348.0,75,4,1797.24,2,4,4,244
3,12349.0,19,1,1757.55,4,1,4,414
4,12350.0,310,1,334.40,1,1,2,112


In [36]:
def rfm_segment(row):
  r = int(row["r_score"])
  f = int(row["f_score"])
  m = int(row["m_score"])

  if r>=4 and f>=4 and m>=4:
    return "Champions"
  if r>=3 and f>=3:
    return "Loyal Customers"
  if r>=4 and f<=2:
    return "New Customers"
  if r==3 and f==3:
    return "Potential Customers"
  if r<=2 and f>=4:
    return "At Risk"
  if r==1 and f<=2:
    return "Lost"
  return "Others"

In [37]:
rfm["segment"] = rfm.apply(rfm_segment, axis=1)

In [38]:
rfm["segment"].value_counts()

,count
segment,
Others,1120
Loyal Customers,998
Champions,962
Lost,664
New Customers,319
At Risk,275


In [39]:
df.to_csv("cleaned_transactions.csv",index=False)

In [40]:
orders.to_csv("orders_table.csv", index=False)

In [41]:
rfm.to_csv("rfm_table.csv", index=False)

In [42]:
orders["invoiceno"] = orders["invoiceno"].astype(str)
df["invoiceno"] = df["invoiceno"].astype(str)

In [43]:
orders.to_csv("orders_table.csv", index=False)
df.to_csv("cleaned_transactions.csv", index=False)